In [1]:
import json

In [2]:
class Content:
    
    MIN_CONSUMEPTION_TIME = 7.0
    MAX_CONSUMEPTION_TIME = 30.0
    
    def __init__(self, id, consumedTime, likeFlag, tags):
        self.id = id
        self.consumedTime = consumedTime
        self.likeFlag = likeFlag
        self.tags = tags
    
    def getId(self):
        return self.id
    
    def getLikeFlag(self):
        return self.likeFlag
        
    def getConsumedTime(self):
        return self.consumedTime
    
    def isConsumed(self):
        return (self.MIN_CONSUMEPTION_TIME<=self.consumedTime) & (self.consumedTime<=self.MAX_CONSUMEPTION_TIME)
    
    def getTags(self):
        return self.tags
    
    def addConsumedTime(self, time):
        if time < self.MIN_CONSUMEPTION_TIME:
            return
        else:
            self.consumedTime += time
            
        if self.consumedTime > self.MAX_CONSUMEPTION_TIME:
            self.consumedTime = self.MAX_CONSUMEPTION_TIME
    
    def initConsumedTime(self):
        self.sonsumedTime = 0
    
    def like(self):
        self.likeFlag = True

    def cancel(self):
            self.likeFlag = False

    def print(self):
        print("__________________________________________")
        print("# create new content and show")
        print("id:       %s"%(self.id))
        print("consumeTime: %f"%(self.consumedTime))
        print("likeFalg: %s"%("True" if self.likeFlag else "False"))
        print("tags:     %s"%(str(self.tags)))
        

    @staticmethod
    def Test(self):
        content = Content("animal_0.jpg", 10.0, False, ["animal"])
        content.print()

In [3]:
class ContentDao:
    
    def __init__(self, path = "./database/data.json"):
        self.path = path
        self.__readDatabase()
    
    def __writeDatabase(self):
        with open(self.path, 'w', encoding='utf-8') as json_db:
            json.dump(self.db, json_db, indent="\t")
            
    def __readDatabase(self):
        with open(self.path, 'r') as json_db:
            self.db = json.load(json_db)
            
    def doesExist(self, id):
        return self.db.get(id) != None
    
    def getContent(self, id):
        if not self.doesExist(id):
            return None
        else:
            c = self.db[id]
            return Content(id, c["consumed_time"], c["like_flag"], c["tags"])
        
    def getAllContents(self):
        contents = []
        keys = self.db.keys()
        for key in keys:
            contents.append(self.getContent(key))
        return contents
    
    def updateContent(self, content):
        self.deleteContent(content.getId())
        self.insertContent(content)
    
    def insertContent(self, content):
        id = content.getId()
        if self.doesExist(id):
            return
        else:
            dict = {}
            dict["consumed_time"] = content.getConsumedTime()
            dict["like_flag"] = content.getLikeFlag()
            dict["tags"] = content.getTags()
            self.db[id] = dict
            self.__writeDatabase()

    def deleteContent(self, id):
        if self.doesExist(id):
            del self.db[id]
            self.__writeDatabase()
    
    def countContent(self):
        return len(list(self.db.keys()))
        
    def print(self):
        print(self.db)
        
    @staticmethod
    def Test():
        print("__________________________________________")
        print("# load db and show")
        cd = ContentDao()
        print("DB 출력")
        cd.print()
        
        print("__________________________________________")
        print("# create new contents and show that")
        content1 = Content("test1", 0, False, ["test1"])
        content2 = Content("test2", 0, False, ["test2"])
        content3 = Content("test3", 0, False, ["test3"])
        content1.print()
        content2.print()
        content3.print()
        
        print("__________________________________________")
        print("# insert contents and count")
        cd.insertContent(content1)
        cd.insertContent(content2)
        cd.insertContent(content3)
        print("count: %d"%cd.countContent())
        
        print("__________________________________________")
        print("# get and show all")
        contents = cd.getAllContents()
        for content in contents:
            content.print()

        print("__________________________________________")
        print("# delete contents and count num")
        cd.deleteContent("test1")
        cd.deleteContent("test2")
        cd.deleteContent("test3")
        print("# delete content and count num")
        print("count: %d"%cd.countContent())

In [4]:
import random

class ContentPriorityManager:
    
    SECTION_SIZE = 3
    def __init__(self, contents):
        self.initContentTable(contents)
        
    def initContentTable(self, contents):
        self.contentStateTable = {}
        for content in contents:
            self.contentStateTable[content.getId()] = "not_sent"

        self.sentContentIdList = []
        self.nextContentIdList = []
        self.notSentContentIdList = list(self.contentStateTable.keys())
        random.shuffle(self.notSentContentIdList)
    
    def getNextContentIdList(self):
        return self.nextContentIdList
    
    def calulatePriorityScore(self, tag):
        likeCount = tag.getLikeCount()
        meanConsumedTime = 0
        if tag.getConsumedCount() != 0:
            meanConsumedTime = tag.getTotalConsumedTime()/tag.getConsumedCount()
        return likeCount*10 + (meanConsumedTime-5)/25*10
    
    def renewNextContentIdList(self, tagPriorityTable, n = SECTION_SIZE):
        # 우선순위 부여
        priority_list = []
        for contentId in self.notSentContentIdList:
            content = ContentDao().getContent(contentId)
            # 부여된 속성 중 가장 선호도가 큰 속성의 선호도 값을 사용
            temp_priority_list = []
            for tagId in content.getTags():
                tag = tagPriorityTable[tagId]
                temp_priority_list.append(self.calulatePriorityScore(tag))
            
            priority_list.append([contentId, max(temp_priority_list)])
            
        priority_list = sorted(priority_list, key = lambda x : x[1], reverse=True) # 두 번째 인자 즉 우선순위에 따른 내림차순정렬
        
        for contentId, priority in priority_list[:n]:
            self.nextContentIdList.append(contentId)
            self.notSentContentIdList.remove(contentId)

    def moveAllNextContentIdsIntoSentIdList(self):
        for contentId in self.nextContentIdList:
            self.sentContentIdList.append(contentId)
            self.nextContentIdList = []
    
    def print(self):
        cst = self.contentStateTable
        for key in cst.keys():
            print("id: %10s     state: %s"%(key, cst[key]))

        print("sentList: %s"%str(self.sentContentIdList))
        print("nextList: %s"%str(self.nextContentIdList))
        print("notSentList: %s"%str(self.notSentContentIdList))
        
    @staticmethod
    def Test():
        cd = ContentDao()
        print("__________________________________________")
        print("# create contents and insert into db and show all")
        cd.insertContent(Content("content1", 5, True, ["tag1"]))
        cd.insertContent(Content("content2", 5, True, ["tag1"]))
        cd.insertContent(Content("content3", 5, True, ["tag1"]))
        cd.insertContent(Content("content4", 10, True, ["tag2"]))
        cd.insertContent(Content("content5", 10, True, ["tag2"]))
        cd.insertContent(Content("content6", 10, True, ["tag2"]))
        cd.insertContent(Content("content7", 10, False, ["tag2"]))
        cd.print()        
        
        
        print("__________________________________________")
        print("# create content priority manager with content list")
        contents = cd.getAllContents()
        cpm = ContentPriorityManager(contents)
        cpm.print()
        

        print("__________________________________________")
        print("# create tag priority manager and show")
        tpm = TagPriorityManager(contents)
        tpm.print()
        
        print("__________________________________________")
        print("# renew next id list with tag priority table and show")
        cpm.renewNextContentIdList(tpm.getTagPriorityTable())
        cpm.print()

        print("__________________________________________")
        print("move next list into sent list and show")
        cpm.moveAllNextContentIdsIntoSentIdList()
        cpm.print()
        
        print("__________________________________________")
        print("delete all inserted content from db")
        for content in contents:
            cd.deleteContent(content.getId())
        

In [5]:
class Tag:
    def __init__(self, id, likeCount = 0, totalConsumedTime = 0.0, consumedCount = 0):
        self.id = id
        self.likeCount = likeCount
        self.totalConsumedTime = totalConsumedTime
        self.consumedCount = consumedCount
        
    def addLikeCount(self, count):
        self.likeCount += count
        
    def subLikeCount(self, count):
        self.likeCount -= count
        
    def addConsumedCount(self, count):
        self.consumedCount += count
        
    def subConsumedCount(self, count):
        self.consumedCount -= count
    
    def addTotalConsumedTime(self, time):
        self.totalConsumedTime += time;
        
    def getId(self):
        return self.id
    
    def getLikeCount(self):
        return self.likeCount
    
    def getTotalConsumedTime(self):
        return self.totalConsumedTime
    
    def getConsumedCount(self):
        return self.consumedCount
        
    def print(self):
        print("id: %s"%(self.id))
        print("likeCount: %s"%(self.likeCount))
        print("ConsumedCount: %s"%(self.totalConsumedTime))
        print("totalConsumedTime: %s"%(self.consumedCount))
        
    @staticmethod
    def Test():
        print("__________________________________________")
        print("# create Tag object and show")
        tag = Tag("test")
        tag.print()

In [6]:
class TagPriorityManager:
    def __init__(self, contents):
        self.initTagPriorityTable(contents)
        
    def initTagPriorityTable(self, contents):
        self.tagPriorityTable = {}
        for content in contents:
            tags = content.getTags()
            for tagId in tags:
                if not self.doesIdExist(tagId):# 태그 정보가 없으면 새로운 초기화된 태그정보 삽입
                    self.tagPriorityTable[tagId] = Tag(tagId)
                    
                # 태그정보 누적
                self.addLikeCountById(tagId, content.getLikeFlag())
                if content.isConsumed():
                    self.addTotalConsumedTimeById(tagId, content.getConsumedTime())
                    self.addConsumedCountById(tagId, int(content.isConsumed()))
            
    def doesIdExist(self, id):
        return self.tagPriorityTable.get(id) != None
        
    def print(self):
        for tagId in self.tagPriorityTable.keys():
            data = self.tagPriorityTable[tagId]
            print("key: %s \t likeCount: %d \t consumeCount: %d \t totalConsumeTime: %5d"%(tagId, 
                                                                                self.getLikeCountById(tagId),
                                                                                self.getConsumedCountById(tagId),
                                                                                self.getTotalConsumedTime(tagId)))
    
    def getLikeCountById(self, id):
        return self.tagPriorityTable[id].getLikeCount()
    
    def getConsumedCountById(self, id):
        return self.tagPriorityTable[id].getConsumedCount()
    
    def getTotalConsumedTime(self, id):
        return self.tagPriorityTable[id].getTotalConsumedTime()
    
    def addLikeCountById(self, id, count):
        self.tagPriorityTable[id].addLikeCount(count)
        
    def subLikeCountById(self, id, count):
        self.tagPriorityTable[id].subLikeCount(count)

    def addConsumedCountById(self, id, count):
        
        self.tagPriorityTable[id].addConsumedCount(count)
        
    def subConsumedCountById(self, id, count):
        self.tagPriorityTable[id].subConsumedCount(count)
    
    def addTotalConsumedTimeById(self, id, time):
        self.tagPriorityTable[id].addTotalConsumedTime(time)
        
    def subTotalConsumedTimeById(self, id, time):
        self.tagPriorityTable[id].subTotalConsumedTime(time)
    
    
    def getTagPriorityTable(self):
        return self.tagPriorityTable
    
    @staticmethod
    def Test():
        content1 = Content("test1", 20, True, ["test1"])
        content2 = Content("test2", 10, True, ["test2"])
        content3 = Content("test3", 4, False, ["test3"])
        
        contents = [content1, content2, content3]
        
        tpm = TagPriorityManager(contents)
        tpm.print()
    

In [7]:
class MainDirector:
    def __init__(self):
        self.cd = ContentDao()
        contents = self.cd.getAllContents()
        self.cpm = ContentPriorityManager(contents)
        self.tpm = TagPriorityManager(contents)
        self.cpm.renewNextContentIdList(self.tpm.getTagPriorityTable())
        
    def getFirstContentSet(self):
        contentIdList = self.cpm.getNextContentIdList()
        contentSet = self.getContentList(contentIdList)
        self.cpm.moveAllNextContentIdsIntoSentIdList()
        self.cpm.renewNextContentIdList(self.tpm.getTagPriorityTable())
        jsonResponse = json.dumps(contentSet)
        return jsonResponse
    
    def getNextContentSet(self, userActions):
        contentIdList = self.cpm.getNextContentIdList()
        contentSet = self.getContentList(contentIdList)
        self.cpm.moveAllNextContentIdsIntoSentIdList()
#         self.handleUserAction(userActions)
        self.cpm.renewNextContentIdList(self.tpm.getTagPriorityTable())
        jsonResponse = json.dumps(contentSet)
        return jsonResponse
    
    def handlePressEvent(self, contentId, flag):
        content = self.cd.getContent(contentId)
        content.likeFlag = flag
        self.cd.updateContent(content)
        ##### 귀찮지만 매우 피곤한 관계로 부하를 허용하여 처리    
        contents = self.cd.getAllContents()
        self.tpm = TagPriorityManager(contents)
    
    def handleConsumEvent(self, contentId, time):
        content = self.cd.getContent(contentId)
        content.addConsumedTime(time)
        self.cd.updateContent(content)
        ##### 귀찮지만 매우 피곤한 관계로 부하를 허용하여 처리
        contents = self.cd.getAllContents()
        self.tpm = TagPriorityManager(contents)

                
    def handleUserAction(self, userActions):
        print("handle")
    
    def getContentList(self, contentIdList):
        contents_dict = {}
        for contentId in contentIdList:
            content = self.cd.getContent(contentId)
            dict = {}
            dict["id"] = content.getId()
            dict["consumed_time"] = content.getConsumedTime()
            dict["like_flag"] = content.getLikeFlag()
            dict["tags"] = content.getTags()
            contents_dict[contentId] = dict
        return contents_dict
    
    @staticmethod
    def Test():
        md = MainDirector()
        nextContent = md.getFirstContentSet()
        print(nextContent)
        nextContent = md.getNextContentSet(None)
        print(nextContent)

In [14]:
# coding = utf-8
 
from flask import Flask, render_template, request
 
md = None 
app = Flask(__name__, template_folder="./")

@app.route("/test", methods=["GET"])
def home():
    return render_template('test.html', name="0.JPG")

@app.route("/consum", methods=["POST"])
def consum():
    global md
    id = request.form["id"]
    time = float(request.form["time"])
    md.handleConsumEvent(id, time)
    md.tpm.print()

@app.route("/press", methods=["POST"])
def press():
    global md
    id = request.form["id"]
    flag = request.form["flag"] == "True"
    print(id, flag)
    md.handlePressEvent(id, flag)

@app.route("/first", methods=["GET"])
def first():
    global md
    md = MainDirector()
    firstSet = md.getFirstContentSet()
    return firstSet

@app.route("/next", methods=["POST"]) 
def next():
    global md
    nextSet = md.getNextContentSet(None)
    return nextSet

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [15]:
# 선호도 초기화 코드
def initPreference():
    cd = ContentDao()
    contents = cd.getAllContents()
    for content in contents:
        content.consumedTime = 0
        content.likeFlag = False
        cd.updateContent(content)
        
initPreference()

In [ ]:
# # Content Test
# print("*************************************************************************** test for Content")
# Content.Test(Content)
# print("SUCCESS!!!!!\n")

# print("*************************************************************************** test for ContentDao")
# ContentDao().Test()
# print("SUCCESS!!!!!\n")

# print("*************************************************************************** test for SendingOrderManager")
# ContentPriorityManager.Test()
# print("SUCCESS!!!!!\n")

# print("*************************************************************************** test for Tag")
# Tag.Test()
# print("SUCCESS!!!!!\n")

# print("*************************************************************************** test for TagPriorityManager")
# TagPriorityManager.Test()
# print("SUCCESS!!!!!\n")